In [1]:
import pandas as pd
import savReaderWriter as spss
from pathlib import Path

In [2]:
df = pd.read_csv('Data files/PISA data/small_stuscho.csv')
country_dict_df = pd.read_csv('Data files/PISA data/nations.csv')

In [3]:
def student2school(df, id):
    '''
        Takes the 'IDs_sorted_by_student.csv' dataframe and student id, returns school id of the student.
        '''
    return student2schoolHelper(df, id, 0, len(df))

In [4]:
def student2schoolHelper(df, id, start, end):
    if start > end:
        print('Student ID:' + id + ' not exists!')
        raise KeyError()
    elif start == end:
        if id == student2schoolHelper['CNTSCHID'][start]:
            return student2schoolHelper['SUBNATIO'][start]
        else:
            print('Student ID:' + id + ' not exists!')
            raise KeyError()
    index = (end + start) // 2
    mid_value = df['CNTSCHID'][index]
    if mid_value == id:
        return df['SUBNATIO'][index]
    elif mid_value > id:
        return student2schoolHelper(df, id, start, index)
    else:
        return student2schoolHelper(df, id, index, end)

In [5]:
def school2nation(id):
    '''
        Takes school id, returns nation id of the school.
        '''
    if id < 97100000:
        return id // 100000 * 10000
    if id < 97200000:
        return 7240000
    return 8400000

In [6]:
def student2nation(id):
    '''
        Takes student id, returns nation id of the student.
        '''
    return school2nation(id)

In [7]:
def info(df, id):
    '''
    :param df: the dataframe of 'nations.csv'
    :param id: the id of nation
    :return: a tuple - (string country name, string country code, string continent)
    '''
    try:
        target = df[df['nationID'] == id]
    except KeyError:
        print('Nation ID:' + id + ' not exists!')
        raise KeyError()

    name = target['nationName'].tolist()
    code = target.nationCode.tolist()
    continent = target.continent.tolist()

    return (name[0], code[0], continent[0])

In [8]:
def infos(country_df, id_list):
    '''
    :param country_df: the dataframe of 'nations.csv'
    :param id_df: the dataframe of country id
    :return: a dataframe - (string country name, string country code, string continent)
    '''

    name_list = []
    continent_list = []
    code_list = []

    for i in range(len(id_list)):
        t = info(country_df, id_list[i])
        name_list.append(t[0])
        code_list.append(t[1])
        continent_list.append(t[2])

    dict = {'CountryName':name_list, 'CountryCode': code_list, 'Continent':continent_list}
    res = pd.DataFrame(dict)
    
    return res

In [10]:
def get_student_info():    
    # Dataframe processing
    df1 = df[['CNTRYID', 'CNTSCHID', 'CNTSTUID', 'ST004D01T']]
    df1.columns = ['CountryID', 'SchoolID', 'StudentID', 'Gender']
    stuid_list = df1['StudentID'].tolist()
    countryid_list = []
    for stuid in stuid_list:
        countryid_list.append(student2nation(stuid))
    df_country = infos(country_dict_df, countryid_list)
    df_math = df.iloc[:, 809:819]
    df_math['Mathematics'] = df_math.mean(axis=1)
    df_read = df.iloc[:, 819:829]
    df_read['Reading'] = df_read.mean(axis=1)
    df_science = df.iloc[:, 829:839]
    df_science['Science'] = df_science.mean(axis=1)
    
    attributes_list = []
    user_input = input('Using default attributes?(y/n) ')
    if user_input == 'y':
        attributes_list = ['IBTEACH', 'WEALTH', 'ESCS',
                           'SC013Q01TA', 'SCIERES']
    elif user_input == 'n':
        print('Input attribute name, "s" to stop:')
        attribute = input()
        while (attribute != 's'):
            if attribute not in df.columns.values:
                raise ValueError('Invalid Input!')
            else:
                attributes_list.append(attribute)
            attribute = input()
    else:
        raise ValueError('Invalid Input!')
        
    df_att = df[attributes_list]
    
    output_df = pd.concat([df1, df_country, df_math['Mathematics'],
                           df_read['Reading'], df_science['Science'],
                           df_att], axis=1, sort=False)
    output_df.loc[df1['Gender'] == 1, 'Gender'] = 'Female'
    output_df.loc[df1['Gender'] == 2, 'Gender'] = 'Male'
    output_df.to_csv('student_info.csv', encoding='utf-8')

In [11]:
get_student_info()

Using default attributes?(y/n) y


In [12]:
result_df = pd.read_csv('student_info.csv')
result_df.head()

,Unnamed: 0,CountryID,SchoolID,StudentID,Gender,CountryName,CountryCode,Continent,Mathematics,Reading,Science,IBTEACH,WEALTH,ESCS,SC013Q01TA,SCIERES
0,0,8,800001,803627,Female,Albania,ALB,EU,453.2652,464.7217,495.2442,,,,1,3
1,1,8,800001,800454,Female,Albania,ALB,EU,420.3666,421.9881,483.8646,,,,1,3
2,2,8,800001,800893,Female,Albania,ALB,EU,352.3811,531.4783,429.1337,,,,1,3
3,3,8,800001,804180,Female,Albania,ALB,EU,399.9742,530.6209,416.9326,,,,1,3
4,4,8,800001,800491,Female,Albania,ALB,EU,317.4811,401.8058,415.8643,,,,1,3
